# 正则查找，标注功能
介绍一个使用双正则表达式筛选一句句子或者一段文本的函数，可以用于文本清洗，或者文本分析的预处理。

In [1]:
import re

def filter_regex_matches(text, primary_regex, remove_regex_list=None) -> list:
    '''
    使用两段正则来搜索文本中的目标关键词
    :param text: 被搜索的文本
    :param primary_regex: 用于搜索目标的正则
    :param remove_regex_list: 用于排除目标的正则
    :return: list的方式返回寻找到的目标关键词
    '''
    try:
        # 使用正则表达式找出所有匹配项
        matches = re.findall(primary_regex, text)

        if remove_regex_list:
            assert type(remove_regex_list) == list, "remove_regex_list必须为列表类型"
            # 移除与remove_regex_list中任一正则表达式匹配的项
            for regex in remove_regex_list:
                matches = [match for match in matches if not re.search(regex, match)]
        # 返回匹配项列表
        return matches

    except Exception as e:
        print(f"filter出现错误:{text} -- {e}")


def sub_regex_matches(text, primary_regex, remove_regex_list=None) -> str:
    '''
    使用两段正则来清除文本中的目标关键词
    :param text: 被搜索的文本
    :param primary_regex: 用于搜索目标的正则
    :param remove_regex_list: 用于排除目标的正则
    :return: 返回被清除干净的文本
    '''
    matches = filter_regex_matches(text, primary_regex, remove_regex_list)
    try:
        for targetWord in matches:
            # 逐个替换匹配对
            patternWord = r"\b{}\b".format(targetWord)
            text = re.sub(patternWord,'',text)
        text = re.sub(r'\s{2,}',' ',text)

        return text.strip()

    except Exception as e:
        print(f"sub出现错误:{text} -- {e}")



def replace_regex_matches(text, primary_regex, Tag ,remove_regex_list=None, search_source=None):
    '''
    使用两段正则寻找文本1(search_source)中的目标关键词,同时对出现在文本2(text)中的目标关键词使用Tag进行标记
    :param text: 被打上Tag的文本2
    :param primary_regex: 用于搜索目标的正则
    :param Tag: 用于标记目标的Tag
    :param remove_regex_list: 用于排除目标的正则
    :param search_source: 用于搜索的文本1
    :return: 返回被清除干净的文本
    '''
    if search_source:
        assert type(search_source) == str
    try:
        # 使用正则表达式找出所有匹配项
        if search_source:
            matches = filter_regex_matches(search_source, primary_regex, remove_regex_list)
        else:
            matches = filter_regex_matches(text, primary_regex, remove_regex_list)
        # 返回匹配项列表
        index_nb = 0
        for match_object in matches: 
            index_nb += 1
            patternWord = r"\b{}\b".format(match_object)
            replace_txt = f"[{Tag}_{index_nb}:{match_object}]"
            text = re.sub(patternWord,replace_txt,text)
        text = re.sub(r'\s{2,}',' ',text)
        return text.strip()

    except Exception as e:
        print(f"replace出现错误:{text} -- {e}")

# 测试标注年份

In [2]:
# 用于识别汽车品牌和年份的字典
from QJBSL.dataset import load_mvl
from QJBSL.Tool.NestedDict import get_nested_ePID_dict

# 读取数据库并且设置Fitment查询顺序为：Make -> Model -> Submodel -> Year
FITMENT_DICT = get_nested_ePID_dict(load_mvl(), col_order = ['Make', 'Model', 'Submodel','Year'])

# 随便从电商网站上找了一个listing的标题作为我们的输入字符串
title_structure = simplest_title = 'NEW 200 AMP Alternator fits Nissan Pathfinder Infiniti G35 QX4 2003-2004'

In [3]:
# 用于识别Year的正则组：
regExp_YEAR = {
    'regExp': r"\b(?:19|20)?\d{2}\s*[-]\s*(?:19|20)?\d{2}\b|\b(?:19|20)\d{2}\b",
    'regExp_excludes':r"\b\d{2}\s*[-]\s*\d{4}\b|\b\d{4}\s*[-]\s*\d{2}\b"
}

# 识别Year
title_structure = (
    replace_regex_matches(title_structure,
                          primary_regex = regExp_YEAR.get('regExp'),
                           Tag = 'YEAR',
                           remove_regex_list = [regExp_YEAR.get('regExp_excludes')],
                            search_source = title_structure)
)
# 展示标记效果
title_structure


'NEW 200 AMP Alternator fits Nissan Pathfinder Infiniti G35 QX4 [YEAR_1:2003-2004]'

# 测试标注汽车品牌

In [4]:
def splice_ReExp_by_wordlist_2(filter_words_list:list) -> re.Pattern:
    '''
    根据给定的词列表制作筛选正则表达式
    :param filter_words_list: 包含筛选词的列表
    :return: 返回制作好的筛选词正则表达式字符串，或者返回 None（如果出现异常）
    -----------------------------------------------------------------------------
    用例:FILTER_WORDS_PATTERN = re.compile(splice_ReExp_by_wordlist(filter_words_list),flags=re.I)
    '''

    # 替换原始string中的标点符号防止错误
    def convert_symbol(match_obj):
        if match_obj.group(1) is not None:
            return r"[{}]".format(match_obj.group(1))

    filter_words_list = [re.sub(r"([^a-zA-Z0-9 \n\.])", convert_symbol, x) for x in filter_words_list if x != '']
    try:
        # 初始化筛选词正则表达式字符串
        return_key_word_string = ''
        # 循环遍历除了最后一个词以外的所有词
        for word in filter_words_list[:-1]:
            # 将当前词添加到筛选词正则表达式字符串中，并用正则表达式标记包围
            return_key_word_string += r"\b{}\b".format(word)
            # 添加一个竖线，以便将每个词的正则表达式字符串组合成一个整体正则表达式
            return_key_word_string += "|"
        # 将最后一个词添加到筛选词正则表达式字符串中，并用正则表达式标记包围
        return_key_word_string += r"\b{}\b".format(filter_words_list[-1])
        # 返回制作好的筛选词正则表达式字符串
        pattern = re.compile(return_key_word_string,flags=re.I)
        return pattern
    except Exception as e:
        # 如果出现异常，输出错误信息，并返回 None
        print("An error occurred in splicing the the long regular expression: ", e)
        return None

In [5]:
# 测试Make和Model相关的识别
# # 制作正则表达式,并替换结构化标题

PATTERN_Ignore_Make = splice_ReExp_by_wordlist_2(FITMENT_DICT.keys())

Make_list = (
    filter_regex_matches(simplest_title, PATTERN_Ignore_Make)
)

title_structure = (
    replace_regex_matches(title_structure, PATTERN_Ignore_Make, 'MAKE')
)
simplest_title = sub_regex_matches(simplest_title, PATTERN_Ignore_Make)

if Make_list:
    Model_list = []
    index_MAKE = 0
    for make in Make_list:
        index_MAKE += 1
        model_tag = f"MAKE_{index_MAKE}-MODEL"

        PATTERN_Ignore_Make_Model = splice_ReExp_by_wordlist_2(
            FITMENT_DICT.get(make.lower()).keys()
        )
        title_structure = (
            replace_regex_matches(title_structure, PATTERN_Ignore_Make_Model, model_tag,search_source=simplest_title)
        )
        Model_list.append(filter_regex_matches(simplest_title, PATTERN_Ignore_Make_Model))
        simplest_title = sub_regex_matches(simplest_title, PATTERN_Ignore_Make_Model)

        if Model_list:
            Submodel_list = []
            index_MODEL = 0
            for model in Model_list:
                index_MODEL += 1
                submodel_tag = f"MAKE_{index_MAKE}-MODEL_{index_MODEL}-SUBMODEL"
                try:
                    PATTERN_Ignore_Make_Model_Submodel = splice_ReExp_by_wordlist_2(
                        FITMENT_DICT
                        .get(make.lower())
                        .get(model.lower()).keys())
                except:
                    PATTERN_Ignore_Make_Model_Submodel = r'Error happens'

                title_structure = (
                    replace_regex_matches(title_structure,
                                    PATTERN_Ignore_Make_Model_Submodel,
                                    submodel_tag,
                                    search_source=simplest_title)
                )
                simplest_title = sub_regex_matches(simplest_title, PATTERN_Ignore_Make_Model_Submodel)

In [6]:
# 检查汽车品牌的标注情况
title_structure
# 这个方法有一个劣势，字典中未保存的关键词不会被识别。随着业务的推进，我们需要积累关键词字典才可以提高准确性。

'NEW 200 AMP Alternator fits [MAKE_1:Nissan] [MAKE_1-MODEL_1:Pathfinder] [MAKE_2:Infiniti] [MAKE_2-MODEL_1:G35] [MAKE_2-MODEL_2:QX4] [YEAR_1:2003-2004]'